In [2]:
import os
from typing import Optional, Any

import torch

from MeloTTS.melo.api import TTS
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

c:\Users\17655\anaconda3\envs\openvoice\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\17655\anaconda3\envs\openvoice\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


d:\OpenVoice
Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [3]:
def load_model(
    ckpt_converter: str,
    device: torch.device,
    language: str = "ZH",
    local_dir=None,
    wav_dir=None,
):
    """
    加载模型: 1. 音色抽取和合成模型; 2. 文字转语音模型
    转音色的功能已注释，如需要则取消注释
    :param: ckpt_converter: 音色抽取和合成模型的权重文件
    :param: device: 模型推理时使用的硬件设备
    :param: language: 设定合成音频时的语言
    """
    tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device, wav_dir=wav_dir)
    tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

    tts_model = TTS(language, device=device, local_dir=local_dir)

    return tone_color_converter, tts_model

In [4]:
root_path = r"D:\OpenVoice"
device = "cuda:0"

print("加载模型中......")
wav_relative_p = "./models--M4869--WavMark/snapshots/0ad3c7b74f641bddb61f6b85cdf2de0d93a5bfef/step59000_snr39.99_pesq4.35_BERP_none0.30_mean1.81_std1.81.model.pkl"
tone_color_converter, tts_model = load_model(
    os.path.join(root_path, "checkpoints_v2/converter"), device=device,
    local_dir=os.path.join(root_path, ".cache"), wav_dir=os.path.join(root_path, ".cache", wav_relative_p),
)
print("成功加载.")

加载模型中......


c:\Users\17655\anaconda3\envs\openvoice\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'D:\OpenVoice\checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []
成功加载.


In [5]:
reference_speaker = os.path.join(root_path, "resources/demo_speaker2.mp3") # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False, cache_dir=os.path.join(root_path, ".cache"))
source_se = torch.load(os.path.join(root_path, f'checkpoints_v2/base_speakers/ses/en-newest.pth'), map_location=device)
print("Real voice loaded.")

OpenVoice version: v2


Estimating duration from bitrate, this may be inaccurate


Real voice loaded.


c:\Users\17655\anaconda3\envs\openvoice\lib\site-packages\torch\functional.py:650: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:868.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [6]:
def synthesis_toncvtr_from_file(
    model: Optional[TTS],
    tone_color_converter: Optional[ToneColorConverter],
    source_se: Any, 
    target_se: Any,
    text_file_path: str,
    output_dir: str,
    speed: float = 1.0
):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    speaker_ids = model.hps.data.spk2id
    default_speaker_id = list(speaker_ids.values())[0]
    
    with open(text_file_path, 'r', encoding='utf-8') as file:
        for line_num, text in enumerate(file.readlines(), start=1):
            text = text.strip()  # Remove trailing newline and whitespace
            if not text:  # Skip empty lines
                continue
            
            # Generate audio using the TTS model
            temp_audio_path = os.path.join(output_dir, f'temp_audio_{line_num}.wav')
            model.tts_to_file(text, default_speaker_id, speed=speed, output_path=temp_audio_path)
            
            # Run the tone color converter
            encode_message = "@MyShell"
            output_audio_path = os.path.join(output_dir, f'output_audio_{line_num}.wav')
            tone_color_converter.convert(
                audio_src_path=temp_audio_path,
                src_se=source_se,
                tgt_se=target_se,
                message=encode_message,
                orig_sr=model.hps.data.sampling_rate,
                output_path=output_audio_path
            )
            
            # Optionally, delete the temporary audio file if no longer needed
            os.remove(temp_audio_path)
            
            print(f"Processed line {line_num}: '{text}' -> {output_audio_path}")



In [9]:
text_file = r"D:\OpenVoice\answer_rest.txt"  # Replace with your text file path
output_directory = r"D:\OpenVoice\output_audio"  # Replace with your desired output directory path
synthesis_toncvtr_from_file(
    model=tts_model,
    tone_color_converter=tone_color_converter,
    source_se=source_se,
    target_se=target_se,
     text_file_path=text_file,
    output_dir=output_directory,
    speed=1.0
)

 > Text split to sentences.
杭电科研实力雄厚, 在众多领域参与并完成了一系列国家“六五”至“十三五”计划重点攻关、“973”“863”等高科技攻关和国家、省部基金科研项目.
2023年实到科研经费突破 7亿元.
近年来, 学校获国家科技进步二等奖4项、国家发明二等奖2项、国家自然科学奖二等奖1项,
其中, 2021年主持获得国家科技进步二等奖1项,
2021年主持获得浙江省科学技术一等奖3项.
学校入选教育部“高等学校科技成果转化和技术转移基地”、科技部“赋予科研人员职务科技成果所有权或长期使用权试点单位”,
以及国家知识产权局、教育部“国家知识产权试点示范高校”,
荣获“全国信息产业科技创新先进集体”称号.
拥有浙江省智慧城市研究中心、浙江省“2011协同创新中心”、浙江省信息化与经济社会发展研究中心、浙江省哲学社科重点研究基地、浙江高等教育研究院、海洋工程研究中心、微电子研究中心、中国财务云服务研究院、生物三维打印与医疗器械研究院、中国科教评价研究院、浙江省管理会计应用创新研究中心、浙江(杭电)创新材料研究院等一批科技教育研究平台.
目前, 学校与国内外数百家企业建立了稳定的科技合作关系,
已成为浙江省科技创新与成果转化的高地,
取得了良好经济效益和社会效益.
 > ===========================


100%|██████████| 12/12 [00:03<00:00,  3.59it/s]


Processed line 1: '杭电科研实力雄厚，在众多领域参与并完成了一系列国家“六五”至“十三五”计划重点攻关、“973”“863”等高科技攻关和国家、省部基金科研项目。2023年实到科研经费突破 7亿元。近年来，学校获国家科技进步二等奖4项、国家发明二等奖2项、国家自然科学奖二等奖1项，其中，2021年主持获得国家科技进步二等奖1项，2021年主持获得浙江省科学技术一等奖3项。学校入选教育部“高等学校科技成果转化和技术转移基地”、科技部“赋予科研人员职务科技成果所有权或长期使用权试点单位”，以及国家知识产权局、教育部“国家知识产权试点示范高校”，荣获“全国信息产业科技创新先进集体”称号。拥有浙江省智慧城市研究中心、浙江省“2011协同创新中心”、浙江省信息化与经济社会发展研究中心、浙江省哲学社科重点研究基地、浙江高等教育研究院、海洋工程研究中心、微电子研究中心、中国财务云服务研究院、生物三维打印与医疗器械研究院、中国科教评价研究院、浙江省管理会计应用创新研究中心、浙江(杭电)创新材料研究院等一批科技教育研究平台。目前，学校与国内外数百家企业建立了稳定的科技合作关系，已成为浙江省科技创新与成果转化的高地，取得了良好经济效益和社会效益。' -> D:\OpenVoice\output_audio\output_audio_1.wav
 > Text split to sentences.
杭电学风浓郁, 学科竞赛成绩突出,
近五年在挑战杯、互联网+、电子设计、数学建模、ACM程序设计和智能汽车等全国大学生顶级权威学科竞赛中获得国家级二等奖以上600余项,
8次入围ACM国际大学生程序设计大赛全球总决赛,
曾获美国数学建模竞赛特等奖等国际奖项,
参加省级以上学科竞赛获奖人数在省属高校中位列首位.
2017年第41届ACM国际大学生程序设计竞赛全球总决赛获得全球排名并列第20；2017年至2022年全国大学生智能汽车竞赛全国总决赛共获一等奖36项（含6项全国冠军）,
总成绩多年居全国第一；2020年全国大学生电子设计竞赛信息科技前沿专题邀请赛获赛事最高奖瑞萨杯；2018年全国大学生数学建模竞赛满额获奖,
总成绩并列全国第一, 2021年斩获数学建模国赛三大奖项之一的“知网研学奖”,
这是我省首次获得该奖项；2018年和2022年两次取得中国大学生服务外包创新创业

 10%|█         | 1/10 [00:00<00:01,  5.55it/s]


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - 'C:\\Users\\17655/nltk_data'
    - 'c:\\Users\\17655\\anaconda3\\envs\\openvoice\\nltk_data'
    - 'c:\\Users\\17655\\anaconda3\\envs\\openvoice\\share\\nltk_data'
    - 'c:\\Users\\17655\\anaconda3\\envs\\openvoice\\lib\\nltk_data'
    - 'C:\\Users\\17655\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
